In [1]:
import pygsheets
from tqdm import tqdm

In [2]:
client = pygsheets.authorize()

### Indonesian Data

In [44]:
id_to_key_ref_ind = {
    'W1': '14dAiIsBLxBUHzXgFDznMjSXCJmFY6qnIWOADqITWx9E',
    'W2': '1jHWRjwkahxpA5T_BWBRaMQ4q9kyg1l0gGz42pC6-dxI',
    'W3': '1C-Yyc17WFWScC5eu5WttQc8P8vCFuLvh8MqlBKiwF8c',
    'W4': '1J0xqeC05H1RVPSERi8fCXWUTGJ4iH3SI0w59BoFh_KU',
    'W5': '1T9jbfP1iapNLS94QZMiYw5K6LrZoP01Qf1jvTWpvdkU',
    'W6': '1G0chmXUmWC-lrZLg9ToQTGCdQmnbP0DA204bIKJn9yc',
}
id_to_key_pred_ind = {
    'W1': '1LIVvmdGV6LHEZDxJjYVre9EkMwJ48tp6mpBPaz7LWoE',
    'W2': '1noJV_ViNfUGabWyWwGChEsVhfPc4ctxh2oVY9kcUaO4',
    'W3': '1M9QwUbbiHkZl_13Y-CNrfzNYN9No5hp_u6KWYhZdi1k',
    'W4': '1gz9l9wsY84cgtKJcYRCCuto7piBpv0DWY2jUDxsndI0',
    'W5': '1iPZV_yL3qwKmSi4mtDwhZNZlB90a63SoFRs2bChVeYc',
    'W6': '1YrprgKaeWU0L4jq78ioL40TyxEC0_nwGroEJVLqhjcg',
}
id_to_key_pred_orig_ind = {
    'W1': '1Wa33qUjeB0pq1QI87jUqHTXlQ88ADCxAFp77vFAhSVU',
    'W2': '1FbYeTu3ZK4vBLoPVpJqCFB8Rj6yzazYRpxRtmPXXf-k',
    'W3': '1UysOeI1QnU8sNXqwQ7FGnULvKCjt8Tm6eoFEdYeQ0xY',
    'W4': '1lKIAWvs9D8JoyNrZJzB6pU4KVtfWstH3tOEfM9dZMDc',
    'W5': '1nQsQPAoxeRr9QzybjrkpTn5-IuBBCByMNN8txkUVV4w',
    'W6': '1SCV9OBxvHwxQ31t6GFirO6et-raMbMpJdfLjn1_HsgU',
}

In [45]:
orig_id = 'W6'
pack_ids = ['W1', 'W2', 'W3', 'W4', 'W5']

sh_orig = client.open_by_key(id_to_key_pred_orig_ind[orig_id])

for pack_id in pack_ids:
    print(f'Reading {pack_id}...')
    wks_orig = sh_orig.worksheet('title', pack_id)
    sh_pack = client.open_by_key(id_to_key_pred_ind[orig_id])
    wks_pack = sh_pack.worksheet('title', pack_id)

    # Get answers
    conflict_stats = wks_pack.get_col(8)[1:]

    # Check answer consistency
    conflict_stats_formatted = []
    const_temp = []
    rows_idx = []
    idx = 1
    for stat in tqdm(conflict_stats):
        idx += 1
        conflict_stat = 'OK' if stat == 'TRUE' else 'CONFLICT'
        const_temp.append(conflict_stat)
        conflict_stats_formatted.append(conflict_stat)
        if len(const_temp) == 5:
            const_temp = []
            rows_idx.append(('G' + str(idx-4), 'G' + str(idx)))

    # Add new colomn & update its values
    wks_orig.insert_cols(6, 1)
    wks_orig.adjust_column_width(7, pixel_size=100)
    wks_orig.set_data_validation('G2', 'G1251')
    wks_orig.update_col(7, ['Answer Status'] + conflict_stats_formatted)

    wks_orig.add_conditional_formatting(start='G2', 
                                     end='G1251', 
                                     condition_type='TEXT_CONTAINS', 
                                     format={'backgroundColor': {'red': 1, 'green': 0.894, 'blue': 0.769, 'alpha': 1}}, 
                                     condition_values=['CONFLICT'])

    requests = []
    for start, end in tqdm(rows_idx):
        requests.append({
            'mergeCells': {
                'range': {
                    'sheetId': wks_orig.id,
                    'startRowIndex': int(start[1:]) - 1,
                    'endRowIndex': int(end[1:]),
                    'startColumnIndex': ord(start[0].upper()) - ord('A'),
                    'endColumnIndex': ord(end[0].upper()) - ord('A') + 1
                },
                'mergeType': 'MERGE_ALL'
            }
        })

    # Perform the batch update
    wks_orig.client.sheet.batch_update(sh_orig.id, requests)

Reading W1...


100%|██████████| 250/250 [00:00<00:00, 135878.71it/s]


Reading W2...


100%|██████████| 250/250 [00:00<00:00, 149689.65it/s]


Reading W3...


100%|██████████| 250/250 [00:00<00:00, 139940.74it/s]


Reading W4...


100%|██████████| 250/250 [00:00<00:00, 126593.75it/s]


Reading W5...


100%|██████████| 250/250 [00:00<00:00, 146224.52it/s]


In [62]:
pred_id = 'W1'
ref_ids = ['W4', 'W5', 'W6']

sh_pred = client.open_by_key(id_to_key_pred_ind[pred_id])

for ref_id in ref_ids:
    print(f'Reading {ref_id}...')
    wks_pred = sh_pred.worksheet('title', ref_id)
    sh_ref = client.open_by_key(id_to_key_ref_ind[ref_id])
    wks_ref = sh_ref.worksheet('title', 'Data')

    # Get answers
    ref_ans = wks_ref.get_col(6)[1:]
    pred_ans = wks_pred.get_col(6)[1:]

    # Check answer consistency
    is_consistent = []
    const_temp = []
    rows_idx = []
    idx = 1
    for pred, ref in tqdm(zip(pred_ans, ref_ans)):
        idx += 1
        consistency = 'TRUE' if pred == ref else 'FALSE'
        const_temp.append(consistency)

        if len(const_temp) == 5:
            if 'FALSE' in const_temp:
                const_temp[0] = 'FALSE'
            for const in const_temp:
                is_consistent.append(const)
            const_temp = []
            rows_idx.append(('H' + str(idx-4), 'H' + str(idx)))

    # Add new colomn & update its values
    wks_pred.insert_cols(6, 2)
    wks_pred.adjust_column_width(7, 8, pixel_size=100)
    wks_pred.set_data_validation('G2', 'G1251', condition_type='BOOLEAN')
    wks_pred.set_data_validation('H2', 'H1251')
    wks_pred.update_col(7, ['Gold Answer'] + ref_ans)
    wks_pred.update_col(8, ['Is Consistent'] + is_consistent)

    wks_pred.add_conditional_formatting(start='H2', 
                                     end='H1251', 
                                     condition_type='TEXT_CONTAINS', 
                                     format={'backgroundColor': {'red': 0.957, 'green': 0.8, 'blue': 0.8, 'alpha': 1}}, 
                                     condition_values=['FALSE'])

    requests = []
    for start, end in tqdm(rows_idx):
        requests.append({
            'mergeCells': {
                'range': {
                    'sheetId': wks_pred.id,
                    'startRowIndex': int(start[1:]) - 1,
                    'endRowIndex': int(end[1:]),
                    'startColumnIndex': ord(start[0].upper()) - ord('A'),
                    'endColumnIndex': ord(end[0].upper()) - ord('A') + 1
                },
                'mergeType': 'MERGE_ALL'
            }
        })

    # Perform the batch update
    wks_pred.client.sheet.batch_update(sh_pred.id, requests)

Reading W4...


1250it [00:00, 627363.89it/s]
100%|██████████| 250/250 [00:00<00:00, 116159.96it/s]


Reading W5...


1250it [00:00, 372363.64it/s]
100%|██████████| 250/250 [00:00<00:00, 128156.44it/s]


Reading W6...


1250it [00:00, 367766.55it/s]
100%|██████████| 250/250 [00:00<00:00, 115851.95it/s]


### Sundanese Data

In [64]:
id_to_key_ref_sun = {
    'W1': '1R-bT3RMu41cx-fnac3mNdb3OT_kb6vof1eMdCbU8_jM',
    'W2': '1ugtKZhO4jLtxW_yEwrrPXZtry2pLaP-6QgiIHKldl5E',
    'W3': '1OzP1XwzU3c-rNXyqxU3JKrrXHeXzSu3D6FIcfym4fHE',
    'W4': '1dNwW9dL4YPBEypUHQ6_-Dse8XL851W0WUiCBb28hRBQ',
    'W5': '16wXGP08nESdLg4am4wfngeMNbK81IJAo3p0udg_iVlU',
    'W6': '1ZQEBOPGAlc6f2IUwavEKpPHY7dQxV4lN5iSQm1WHnX4',
}
id_to_key_pred_sun = {
    'W1': '1MD7saLQYAXCLnpTA_WaV1oVv2vwbBn5hFfJ3z1s-Npg',
    'W2': '1Tn6nttAU3wAiMaJaBb6-P39IJ2sElMNS09bXUo-vI-Q',
    'W3': '1AQG9V74TxDkQAUt5aRs9mNa2fiyLMIML65X8nNQ-WuU',
    'W4': '1vSIJikiPedE4GNUB-zno2SN-L7c_e0IJMcrP8nq2RFk',
    'W5': '1b7G4c2PejzDpSqr4Nxo9sWYiAuE_egJS35Dt1HUkDvY',
    'W6': '1nu3ksVRHl-bTO0LW6yXIHGNhs7iJJ5AsdmHcrb2KxL0',
}
id_to_key_pred_orig_sun = {
    'W1': '1-FLTsgge53Wgb3HmIlM-oCOWow4_kLEJ0bey8MaMxVI',
    'W2': '1-WLiRHFXlD5BawHdBkI2kLSlLttBXU4ufnTwLB8pufM',
    'W3': '1EVv6ktg-6ZC5e9UBFvPIVrZI0T2WAbx3OHau_T-OgSE',
    'W4': '1XyXhn_R3VuNcsmHCmCEwojFM1hUPGhQe6FDrfo3ALds',
    'W5': '1YsreB2g0AeDbiFOIu2JOUmbAkwugBRaaf8SMUA2mrg0',
    'W6': '1oU4K52UaKJT3EEvDdOs8o4tD97Wxl4SKSFQK193w2bQ',
}

In [65]:
orig_id = 'W6'
pack_ids = ['W1', 'W2', 'W3', 'W4', 'W5']

sh_orig = client.open_by_key(id_to_key_pred_orig_sun[orig_id])

for pack_id in pack_ids:
    print(f'Reading {pack_id}...')
    wks_orig = sh_orig.worksheet('title', pack_id)
    sh_pack = client.open_by_key(id_to_key_pred_sun[orig_id])
    wks_pack = sh_pack.worksheet('title', pack_id)

    # Get answers
    conflict_stats = wks_pack.get_col(9)[1:]

    # Check answer consistency
    conflict_stats_formatted = []
    const_temp = []
    rows_idx = []
    idx = 1
    for stat in tqdm(conflict_stats):
        idx += 1
        conflict_stat = 'CONFLICT' if stat == 'FALSE' else 'OK'
        const_temp.append(conflict_stat)
        conflict_stats_formatted.append(conflict_stat)
        if len(const_temp) == 5:
            const_temp = []
            rows_idx.append(('H' + str(idx-4), 'H' + str(idx)))

    # Add new colomn & update its values
    wks_orig.insert_cols(7, 1)
    wks_orig.adjust_column_width(8, pixel_size=100)
    wks_orig.set_data_validation('H2', 'H1251')
    wks_orig.update_col(8, ['Answer Status'] + conflict_stats_formatted)

    wks_orig.add_conditional_formatting(start='H2', 
                                     end='H1251', 
                                     condition_type='TEXT_CONTAINS', 
                                     format={'backgroundColor': {'red': 1, 'green': 0.894, 'blue': 0.769, 'alpha': 1}}, 
                                     condition_values=['CONFLICT'])

    requests = []
    for start, end in tqdm(rows_idx):
        requests.append({
            'mergeCells': {
                'range': {
                    'sheetId': wks_orig.id,
                    'startRowIndex': int(start[1:]) - 1,
                    'endRowIndex': int(end[1:]),
                    'startColumnIndex': ord(start[0].upper()) - ord('A'),
                    'endColumnIndex': ord(end[0].upper()) - ord('A') + 1
                },
                'mergeType': 'MERGE_ALL'
            }
        })

    # Perform the batch update
    wks_orig.client.sheet.batch_update(sh_orig.id, requests)

Reading W1...


100%|██████████| 250/250 [00:00<00:00, 146531.02it/s]


Reading W2...


100%|██████████| 250/250 [00:00<00:00, 244651.42it/s]


Reading W3...


100%|██████████| 250/250 [00:00<00:00, 129023.75it/s]


Reading W4...


100%|██████████| 250/250 [00:00<00:00, 128297.57it/s]


Reading W5...


100%|██████████| 250/250 [00:00<00:00, 130696.25it/s]


In [58]:
pred_id = 'W6'
ref_ids = ['W1', 'W2', 'W3', 'W4', 'W5']

sh_pred = client.open_by_key(id_to_key_pred_sun[pred_id])

for ref_id in ref_ids:
    print(f'Reading {ref_id}...')
    wks_pred = sh_pred.worksheet('title', ref_id)
    sh_ref = client.open_by_key(id_to_key_ref_sun[ref_id])
    wks_ref = sh_ref.worksheet('title', 'Data')

    # Get answers
    ref_ans = wks_ref.get_col(7)[1:]
    pred_ans = wks_pred.get_col(7)[1:]

    # Check answer consistency
    is_consistent = []
    const_temp = []
    rows_idx = []
    idx = 1
    for pred, ref in tqdm(zip(pred_ans, ref_ans)):
        idx += 1
        consistency = 'TRUE' if pred == ref else 'FALSE'
        const_temp.append(consistency)

        if len(const_temp) == 5:
            if 'FALSE' in const_temp:
                const_temp[0] = 'FALSE'
            for const in const_temp:
                is_consistent.append(const)
            const_temp = []
            rows_idx.append(('I' + str(idx-4), 'I' + str(idx)))

    # Add new colomn & update its values
    wks_pred.insert_cols(7, 2)
    wks_pred.adjust_column_width(8, 9, pixel_size=100)
    wks_pred.set_data_validation('H2', 'H1251', condition_type='BOOLEAN')
    wks_pred.set_data_validation('I2', 'I1251')
    wks_pred.update_col(8, ['Gold Answer'] + ref_ans)
    wks_pred.update_col(9, ['Is Consistent'] + is_consistent)

    wks_pred.add_conditional_formatting(start='I2', 
                                        end='I1251', 
                                        condition_type='TEXT_CONTAINS', 
                                        format={'backgroundColor': {'red': 0.957, 'green': 0.8, 'blue': 0.8, 'alpha': 1}}, 
                                        condition_values=['FALSE'])

    requests = []
    for start, end in tqdm(rows_idx):
        requests.append({
            'mergeCells': {
                'range': {
                    'sheetId': wks_pred.id,
                    'startRowIndex': int(start[1:]) - 1,
                    'endRowIndex': int(end[1:]),
                    'startColumnIndex': ord(start[0].upper()) - ord('A'),
                    'endColumnIndex': ord(end[0].upper()) - ord('A') + 1
                },
                'mergeType': 'MERGE_ALL'
            }
        })

    # Perform the batch update
    wks_pred.client.sheet.batch_update(sh_pred.id, requests)

Reading W1...


1250it [00:00, 365739.80it/s]
100%|██████████| 250/250 [00:00<00:00, 119291.92it/s]


Reading W2...


1250it [00:00, 329512.92it/s]
100%|██████████| 250/250 [00:00<00:00, 121785.83it/s]


Reading W3...


1250it [00:00, 372310.75it/s]
100%|██████████| 250/250 [00:00<00:00, 108357.55it/s]


Reading W4...


1250it [00:00, 354320.47it/s]
100%|██████████| 250/250 [00:00<00:00, 115177.50it/s]


Reading W5...


1250it [00:00, 376292.26it/s]
100%|██████████| 250/250 [00:00<00:00, 120387.60it/s]
